In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np

Loading Training from the CSV file

In [4]:
df = pd.read_csv('../data/raw/beer_reviews.csv')

Showing the imported data

In [5]:
df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


Removing unwanted columns including ID column, resetting indexes and writing data to a file

In [6]:
df_cleaned = df.copy()
df_cleaned.drop(['brewery_id', 'review_time', 'review_overall', 'review_profilename', 'beer_name', 'beer_beerid'], axis=1, inplace=True) # Dropping unwanted columns
df_cleaned.dropna(inplace=True) # Remove missing values
df_cleaned.reset_index(drop=True, inplace=True)
df_cleaned.to_csv('../data/interim/beer_reviews.csv', index=False) # Saving intermediate date to a file

Normalising numeric and categorical columns. Saving standard scaler and categorical value order for API to reuse to normalise the requests.

In [8]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from joblib import dump, load # To save the standard scaler

# Normalising numerical values
num_cols = ['review_aroma', 'review_appearance', 'review_palate', 'review_taste', 'beer_abv']
sc = StandardScaler()
X_train_cleaned = df_cleaned.copy()
X_train_cleaned[num_cols] = sc.fit_transform(X_train_cleaned[num_cols])
dump(sc, '../data/processed/std_scaler.bin', compress=True) # Saving standard scaler to use in API

# Converting categorical values to numeric and saving the categories
cat_cols = ['brewery_name']
oe_cat = OrdinalEncoder()
X_train_cleaned[cat_cols] = oe_cat.fit_transform(X_train_cleaned[cat_cols])
brewery_names = oe_cat.categories_[0]
np.save('../data/processed/brewery_names.npy', brewery_names) # Saving unique name array to use in API

# Convert target values to numeric and saving them
target_col = ['beer_style']
oe_target = OrdinalEncoder()
X_train_cleaned[target_col] = oe_target.fit_transform(X_train_cleaned[target_col])
beer_style_names = oe_target.categories_[0]
np.save('../data/processed/beer_style_names.npy', beer_style_names) # Saving unique array to use in API

X_train_cleaned.reset_index(drop=True, inplace=True)

Check normalised data

In [11]:
X_train_cleaned.head()

,brewery_name,review_aroma,review_appearance,beer_style,review_palate,review_taste,beer_abv
0,4886.0,-2.511041,-2.198046,65.0,-3.317269,-3.161958,-0.879384
1,4886.0,-1.792025,-1.384166,51.0,-1.109371,-1.103408,-0.362706
2,4886.0,-1.792025,-1.384166,59.0,-1.109371,-1.103408,-0.233537
3,4886.0,-1.073010,-0.570285,61.0,-1.845337,-1.103408,-0.879384
4,1360.0,1.084037,0.243595,9.0,0.362561,0.955143,0.283141


In [14]:
X_train_cleaned.describe()

,brewery_name,review_aroma,review_appearance,beer_style,review_palate,review_taste,beer_abv
count,1.518814e+06,1.518814e+06,1.518814e+06,1.518814e+06,1.518814e+06,1.518814e+06,1.518814e+06
mean,2.619459e+03,1.109312e-16,3.026282e-16,4.221748e+01,2.271770e-16,5.688781e-17,2.230601e-17
std,1.557927e+03,1.000000e+00,1.000000e+00,3.303752e+01,1.000000e+00,1.000000e+00,1.000000e+00
min,0.000000e+00,-3.949072e+00,-6.267448e+00,0.000000e+00,-4.053235e+00,-3.848142e+00,-3.027902e+00
25%,1.216000e+03,-3.539941e-01,-5.702854e-01,1.200000e+01,-3.734048e-01,-4.172240e-01,-7.932707e-01
50%,2.545000e+03,3.650216e-01,2.435949e-01,3.100000e+01,3.625612e-01,2.689595e-01,-2.335365e-01
75%,4.136000e+03,3.650216e-01,2.435949e-01,7.400000e+01,3.625612e-01,9.551430e-01,6.275930e-01
max,5.154000e+03,1.803053e+00,1.871356e+00,1.030000e+02,1.834493e+00,1.641327e+00,2.181138e+01


Creating a dataframe with both features and target

In [15]:
X = X_train_cleaned[num_cols+cat_cols+target_col]

In [16]:
X.shape

(1518814, 7)

In [17]:
X.head()

,review_aroma,review_appearance,review_palate,review_taste,beer_abv,brewery_name,beer_style
0,-2.511041,-2.198046,-3.317269,-3.161958,-0.879384,4886.0,65.0
1,-1.792025,-1.384166,-1.109371,-1.103408,-0.362706,4886.0,51.0
2,-1.792025,-1.384166,-1.109371,-1.103408,-0.233537,4886.0,59.0
3,-1.073010,-0.570285,-1.845337,-1.103408,-0.879384,4886.0,61.0
4,1.084037,0.243595,0.362561,0.955143,0.283141,1360.0,9.0


In [16]:
import torch
from src.models.pytorch import get_device
from src.data.sets import split_sets_by_time
from src.models.pytorch import PytorchDataset1

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, y_train, X_val, y_val, X_test, y_test = split_sets_by_time(X, target_col='beer_style', test_ratio=0.2)

In [17]:
train_dataset = PytorchDataset1(X_train, y_train)
test_dataset = PytorchDataset1(X_test, y_test)
valid_dataset = PytorchDataset1(X_val, y_val)

In [18]:
X_train.shape

(911290, 6)

In [19]:
y_train.shape

(911290,)

In [20]:
from src.models.null import NullModel

baseline_model = NullModel(target_type='classification')
y_base = baseline_model.fit_predict(y_train)

In [38]:
import torch
import torch.nn as nn
from src.models.pytorch import PytorchMultiClass
from src.models.pytorch import train_classification1, test_classification1
from src.models.pytorch import train_classification, test_classification
from src.models.pytorch import get_device

learning_rate = 0.1
N_EPOCHS = 50
BATCH_SIZE = 18225

model = PytorchMultiClass(X_train.shape[1], num_classes)
device = get_device()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification1(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_acc = test_classification1(valid_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(valid)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')

Epoch: 0
	(train)	|	Loss: 0.0003	|	Acc: 6.5%
	(valid)	|	Loss: 0.0002	|	Acc: 6.2%
Epoch: 1
	(train)	|	Loss: 0.0003	|	Acc: 8.2%
	(valid)	|	Loss: 0.0002	|	Acc: 6.2%
Epoch: 2
	(train)	|	Loss: 0.0003	|	Acc: 8.3%
	(valid)	|	Loss: 0.0002	|	Acc: 6.2%
Epoch: 3
	(train)	|	Loss: 0.0003	|	Acc: 8.3%
	(valid)	|	Loss: 0.0002	|	Acc: 6.2%
Epoch: 4
	(train)	|	Loss: 0.0003	|	Acc: 8.3%
	(valid)	|	Loss: 0.0002	|	Acc: 6.2%
Epoch: 5
	(train)	|	Loss: 0.0003	|	Acc: 8.3%
	(valid)	|	Loss: 0.0002	|	Acc: 6.2%
Epoch: 6
	(train)	|	Loss: 0.0003	|	Acc: 8.3%
	(valid)	|	Loss: 0.0002	|	Acc: 6.2%
Epoch: 7
	(train)	|	Loss: 0.0003	|	Acc: 8.3%
	(valid)	|	Loss: 0.0002	|	Acc: 6.2%
Epoch: 8
	(train)	|	Loss: 0.0003	|	Acc: 8.3%
	(valid)	|	Loss: 0.0002	|	Acc: 6.2%
Epoch: 9
	(train)	|	Loss: 0.0003	|	Acc: 8.3%
	(valid)	|	Loss: 0.0002	|	Acc: 6.2%
Epoch: 10
	(train)	|	Loss: 0.0003	|	Acc: 8.3%
	(valid)	|	Loss: 0.0002	|	Acc: 6.2%
Epoch: 11
	(train)	|	Loss: 0.0003	|	Acc: 8.3%
	(valid)	|	Loss: 0.0002	|	Acc: 6.2%
Epoch: 12
	(train)	|	Loss:

In [39]:
torch.save(model, "../models/pytorch_classification_v1.pt")

In [45]:
test_loss, test_acc = test_classification1(test_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)
#print(f'\tLoss: {test_loss:.4f}\t|\tAccuracy: {test_acc:.1f}')
print(f'\t(Test)\t|\tLoss: {test_loss:.4f}\t|\tAccuracy: {test_acc * 100:.1f}%')

	(Test)	|	Loss: 0.0002	|	Accuracy: 5.3%
